In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install langchain
#!pip install torch
#!pip install sentence_transformers
#!pip install faiss-cpu
#!pip install huggingface-hub
#!pip install pypdf
#!pip -q install accelerate
#!pip install llama-cpp-python
#!pip -q install git+https://github.com/huggingface/transformers
#!pip install unstructured
#!pip install pdf2image
#!pip install pdfminer
#!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 36.7 MB/s eta 0:00:00


In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader, PyMuPDFLoader

In [ ]:
# loading pdf files
loader = PyPDFDirectoryLoader("/content/sample_data/Data/")
data = loader.load()

In [ ]:
print(data)

[Document(page_content='JSS\n Journal of Statistical Software\nNovember 2004, Volume 11, Issue 10. http://www.jstatsoft.org/\nEconometric Computing with HC and HAC\nCovariance Matrix Estimators\nAchim Zeileis\nWirtschaftsuniversit ¨at Wien\nAbstract\nData described by econometric models typically contains autocorrelation and/or het-\neroskedasticity of unknown form and for inference in such models it is essential to use\ncovariance matrix estimators that can consistently estimate the covariance of the model\nparameters. Hence, suitable heteroskedasticity consistent (HC) and heteroskedasticity\nand autocorrelation consistent (HAC) estimators have been receiving attention in the\neconometric literature over the last 20 years. To apply these estimators in practice,\nan implementation is needed that preferably translates the conceptual properties of the\nunderlying theoretical frameworks into computational tools. In this paper, such an imple-\nmentation in the package sandwich in the Rsyst

In [ ]:
# splitting the extracted data into text chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

17

In [ ]:
text_chunks[2]

Document(page_content='Journal of Statistical Software 3\nHC and HAC covariances matrices can now be extracted from the same ﬁtted models using\nvcovHC andvcovHAC . Due to the object orientation of R, these functions are not only limited to\nthe linear regression model but can be easily extended to other models. The HAC estimators\nare already available for generalized linear models (ﬁtted by glm) and robust regression (ﬁtted\nbyrlmin package MASS ). Another important feature of Rthat is used repeatedly below is\nthat functions are ﬁrst-class objects—i.e., functions can take functions as arguments and\nreturn functions—which is particularly useful for deﬁning certain procedures for data-driven\ncomputations such as the deﬁnition of the structure of covariance matrices in HC estimation\nand weighting schemes for HAC estimation.\nThe remainder of this paper is structured as follows: To ﬁx notations, Section 2describes\nthe linear regression model used and motivates the following sections

In [ ]:
# downloading the embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# creating embeddings for each of the text chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
# importing model
llm = LlamaCpp(
    streaming = True,
    model_path = "/content/drive/MyDrive/Models/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature = 0.75,
    top_p = 1,
    verbose = True,
    n_ctx = 4096
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k":2}))

In [ ]:
query = "What is linear regression  model"

In [ ]:
qa.run(query)

' Linear regression is a statistical method for modeling the relationship between dependent variable yi and independent variables xi. It is done by estimating the coefficients of the linear equation that minimizes the sum of squared errors between the observed values and predicted values of yi. The residuals (u) are assumed to have zero mean and constant variance, which allows for OLS estimation of the coefficients β.'